In [ ]:
!pip install accelerate
!pip install nilearn
!pip install surfplot
!pip install neuromaps
!pip install himalaya
!pip install voxelwise_tutorials


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfully uninstalled requests-2.32.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
python-gitlab 4.6.0 requires requests>=2.32.0, but you have requests 2.31.0 which is incompatible.


In [ ]:
import torch
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
from os.path import exists
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt


import nilearn
from nilearn.datasets import fetch_atlas_harvard_oxford, fetch_atlas_schaefer_2018
from nilearn.maskers import NiftiLabelsMasker
from nilearn.plotting import plot_stat_map

from surfplot import Plot
from neuromaps.datasets import fetch_fslr
from neuromaps.transforms import mni152_to_fslr


import sklearn

from himalaya.kernel_ridge import KernelRidgeCV
from himalaya.scoring import correlation_score
from voxelwise_tutorials.delayer import Delayer


from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.pipeline import make_pipeline

from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

import nibabel as nib

/usr/local/lib/python3.10/dist-packages/brainspace/plotting/base.py:40: UserWarning: Using Panel interactively in Colab notebooks requires the jupyter_bokeh package to be installed. Install it with:

    !pip install jupyter_bokeh

and try again.
  pn.extension('vtk')


In [ ]:
device = (
    'cuda'
    if torch.cuda.is_available()
    else 'mps'
    if torch.backends.mps.is_available()
    else 'cpu'
)

print(f"Running on {device.upper()}")

Running on CUDA


In [ ]:
testX = np.array(np.random.rand(500, 120))
testy = np.array(np.random.rand(500, 3))

In [ ]:
testX

array([[0.1810536 , 0.03460911, 0.45542681, ..., 0.64573546, 0.11058279,
        0.62267451],
       [0.33531167, 0.41160918, 0.60790413, ..., 0.20297579, 0.7317685 ,
        0.97443594],
       [0.5182785 , 0.54314879, 0.18778045, ..., 0.44912848, 0.83674693,
        0.3852577 ],
       ...,
       [0.62245954, 0.84252199, 0.07119039, ..., 0.61433674, 0.29988875,
        0.37493421],
       [0.88968242, 0.48789512, 0.44245558, ..., 0.15732841, 0.86521139,
        0.53602893],
       [0.64885161, 0.98424317, 0.84932661, ..., 0.71526369, 0.90049213,
        0.07409635]])

In [ ]:
testy

array([[0.66111108, 0.13018216, 0.93242675],
       [0.07869242, 0.61980934, 0.47628745],
       [0.53947579, 0.97783152, 0.25784477],
       ...,
       [0.0897472 , 0.36249184, 0.95684861],
       [0.78698119, 0.88393373, 0.43679776],
       [0.62777599, 0.62739057, 0.5938426 ]])

In [ ]:
correlation_score(testX, testy[:, 1].reshape(-1,1)).mean()

0.0024405291461550983

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import nibabel as nib
from nilearn.input_data import NiftiLabelsMasker
from nilearn.datasets import fetch_atlas_schaefer_2018
import numpy as np
JHU_atlas = '/content/drive/MyDrive/SharedDataonLLM/jhu.nii'


mask = NiftiLabelsMasker(labels_img=JHU_atlas, standardize=True)

data_path = '/content/drive/MyDrive/SharedDataonLLM/sub127_black+forgot+piemanpni+bronx.nii.gz'

fmri_img = nib.load(data_path)
fmri_data = fmri_img.get_fdata()

chunk_size = 100  # This can be adjusted based on your memory capacity

n_time_points = fmri_data.shape[-1]

all_signals = []

for start in range(0, n_time_points, chunk_size):
    end = min(start + chunk_size, n_time_points)
    fmri_chunk = nib.Nifti1Image(fmri_data[..., start:end], affine=fmri_img.affine)
    signals_chunk = mask.fit_transform(fmri_chunk)
    all_signals.append(signals_chunk)

parcel_signals = np.concatenate(all_signals, axis=0)

print('Shape of the extracted signals:', parcel_signals.shape)


Shape of the extracted signals: (1709, 189)


In [ ]:
import pandas as pd
path_to_com = '/content/drive/MyDrive/SharedDataonLLM/black+forgot+piemanpni+bronx_stories.csv'
allStories = pd.read_csv(path_to_com)
allStories

def get_stimulusData(stories):
    my_dict = {}
    for story in stories:
        temp = allStories[allStories['story'] == story].reset_index(drop=True)
        temp_short = pd.DataFrame(
            temp[['onset', 'offset', 'story', 'word', 'lemma', 'PoS_tag_full']],
            columns=['onset', 'offset', 'story', 'word', 'lemma', 'PoS_tag_full']
        )
        my_dict[story] = temp_short
    return my_dict

stories = ['black', 'forgot', 'piemanpni', 'bronx']

seen = get_stimulusData(stories)

black = pd.DataFrame(seen['black'])
forgot = pd.DataFrame(seen['forgot'])
piemanpni = pd.DataFrame(seen['piemanpni'])
bronx = pd.DataFrame(seen['bronx'])

print(black)
print(forgot)
print(piemanpni)
print(bronx)

           onset      offset  story    word   lemma PoS_tag_full
0       0.240000    0.630000  black      so      so           RB
1       0.680000    1.260000  black       i       I          PRP
2       1.960000    2.300000  black     was      be          VBD
3       2.300000    2.450000  black       a       a           DT
4       2.460000    3.140000  black  junior  junior           NN
...          ...         ...    ...     ...     ...          ...
1539  790.779999  791.009999  black    what    what           WP
1540  791.010000  791.130000  black       i       I          PRP
1541  791.129999  791.469999  black      do      do           VB
1542  792.350000  792.610000  black   thank   thank           NN
1543  799.810000  799.830000  black     you     you          PRP

[1544 rows x 6 columns]
           onset      offset   story   word  lemma PoS_tag_full
0       0.850000    0.980000  forgot   this   this           DT
1       0.980000    1.130000  forgot     is     be          VBZ
2  

In [ ]:
display(black)
display(forgot)
display(piemanpni)
display(bronx)

,onset,offset,story,word,lemma,PoS_tag_full
0,0.240000,0.630000,black,so,so,RB
1,0.680000,1.260000,black,i,I,PRP
2,1.960000,2.300000,black,was,be,VBD
3,2.300000,2.450000,black,a,a,DT
4,2.460000,3.140000,black,junior,junior,NN
...,...,...,...,...,...,...
1539,790.779999,791.009999,black,what,what,WP
1540,791.010000,791.130000,black,i,I,PRP
1541,791.129999,791.469999,black,do,do,VB
1542,792.350000,792.610000,black,thank,thank,NN


,onset,offset,story,word,lemma,PoS_tag_full
0,0.850000,0.980000,forgot,this,this,DT
1,0.980000,1.130000,forgot,is,be,VBZ
2,1.130000,1.200000,forgot,a,a,DT
3,1.200000,1.770000,forgot,story,story,NN
4,2.210000,2.380000,forgot,that,that,IN
...,...,...,...,...,...,...
2130,818.280000,818.710000,forgot,c,c,NNS
2131,819.809999,820.159999,forgot,d,d,NN
2132,821.540000,821.880000,forgot,e,e,NN
2133,822.969999,823.429999,forgot,f,f,NN


,onset,offset,story,word,lemma,PoS_tag_full
0,8.71,8.84,piemanpni,i,I,PRP
1,8.99,9.07,piemanpni,began,begin,VBD
2,9.07,9.24,piemanpni,my,my,PRP$
3,9.63,10.08,piemanpni,illustrious,illustrious,JJ
4,10.08,10.62,piemanpni,career,career,NN
...,...,...,...,...,...,...
987,407.41,407.61,piemanpni,tell,tell,NN
988,407.61,407.77,piemanpni,me,me,PRP
989,407.77,408.00,piemanpni,all,all,DT
990,408.00,408.38,piemanpni,about,about,IN


,onset,offset,story,word,lemma,PoS_tag_full
0,15.19,15.41,bronx,when,when,WRB
1,15.41,15.46,bronx,i,I,PRP
2,15.46,15.66,bronx,get,get,VB
3,15.66,15.76,bronx,the,the,DT
4,15.83,16.23,bronx,baton,baton,NN
...,...,...,...,...,...,...
1374,549.44,549.46,bronx,of,of,IN
1375,549.63,549.84,bronx,her,her,PRP$
1376,549.85,549.86,bronx,highbridge,NaN,NaN
1377,550.32,550.34,bronx,her,her,PRP$


In [ ]:


!pip install --upgrade huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.6/402.6 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
MAMBA130 = "state-spaces/mamba-370m-hf"
GPT2 = "gpt2"
BERT = "google-bert/bert-base-uncased"
OPT = "facebook/opt-2.7b"
QWEN = "Qwen/Qwen1.5-0.5B-Chat"
STABLELM2 = "stabilityai/stablelm-2-1_6b"
FLAMINGO = "HuggingFaceM4/idefics-9b-instruct" #Open Source Reproduction but not the official released one
model = AutoModelForCausalLM.from_pretrained(MAMBA130, trust_remote_code=True)
autoToken = AutoTokenizer.from_pretrained(MAMBA130, trust_remote_code=True)

config.json:   0%|          | 0.00/917 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.49G [00:00<?, ?B/s]

The fast path is not available because on of `(selective_state_update, selective_scan_fn, causal_conv1d_fn, causal_conv1d_update, mamba_inner_fn)` is None. Falling back to the naive implementation. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d


generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.79k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model.eval()

In [ ]:
model?

In [ ]:
story_df = pd.concat([black, forgot, piemanpni, bronx], axis=0, ignore_index=True)

display(story_df)

story_df.insert(0, 'w_id', story_df.index.values)

story_df['token'] = story_df.word.apply(autoToken.tokenize)
story_df = story_df.explode('token', ignore_index=True)

story_df['token_id'] = story_df.token.apply(autoToken.convert_tokens_to_ids)
tokenId_lst = story_df.token_id.tolist()

display(story_df)

,onset,offset,story,word,lemma,PoS_tag_full
0,0.24,0.63,black,so,so,RB
1,0.68,1.26,black,i,I,PRP
2,1.96,2.30,black,was,be,VBD
3,2.30,2.45,black,a,a,DT
4,2.46,3.14,black,junior,junior,NN
...,...,...,...,...,...,...
6045,549.44,549.46,bronx,of,of,IN
6046,549.63,549.84,bronx,her,her,PRP$
6047,549.85,549.86,bronx,highbridge,NaN,NaN
6048,550.32,550.34,bronx,her,her,PRP$


,w_id,onset,offset,story,word,lemma,PoS_tag_full,token,token_id
0,0,0.24,0.63,black,so,so,RB,so,601
1,1,0.68,1.26,black,i,I,PRP,i,74
2,2,1.96,2.30,black,was,be,VBD,was,4238
3,3,2.30,2.45,black,a,a,DT,a,66
4,4,2.46,3.14,black,junior,junior,NN,jun,30986
...,...,...,...,...,...,...,...,...,...
7368,6047,549.85,549.86,bronx,highbridge,NaN,NaN,high,8656
7369,6047,549.85,549.86,bronx,highbridge,NaN,NaN,bridge,8298
7370,6048,550.32,550.34,bronx,her,her,PRP$,her,379
7371,6049,550.34,550.46,bronx,bronx,Bronx,NNP,bron,40742


In [ ]:
model_len = autoToken.model_max_length
print(model_len)

if int(model_len//1e30) == 1:
  model_len = 1024
  print("There was no provided max model length, thus the value has been set to 1024")
tensorList = []
torch_tensors = torch.tensor(tokenId_lst, dtype=torch.long)
tensorList.append(torch_tensors[0:model_len])
print(tensorList)
print(len(tensorList))

for i in range(model_len + 1, len(tokenId_lst)+1):
    tensorList.append(torch_tensors[i-model_len:i])

1000000000000000019884624838656
There was no provided max model length, thus the value has been set to 1024
[tensor([ 601,   74, 4238,  ...,  360,  531, 1171])]
1


In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 3.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()
accelerator.free_memory()

model = model.to(device)

In [ ]:
batch_size = 4
layer = 10

embeddings = []
top_guesses = []
ranks = []
true_probs = []
entropies = []


with torch.no_grad():
    data_loader = torch.utils.data.DataLoader(tensorList, batch_size=batch_size, shuffle=False)

    for i, batch in enumerate(tqdm(data_loader)):
        output = model(batch.to(device), output_hidden_states=True)
        logits = output.logits  # torch.Size([2, 1024, 50257])
        states = output.hidden_states[layer]

        if i == 0:
            true_ids = batch[0, :]
            brange = list(range(len(true_ids)-1))
            logits_order = logits[0].argsort(descending=True, dim=-1)
            batch_top_guesses = logits_order[:-1, 0]
            batch_ranks = torch.eq(logits_order[:-1],
                                   true_ids.reshape(-1,1)[1:].to(device)).nonzero()[:, 1]
            batch_probs = logits[0, :-1].softmax(-1)
            batch_probs = batch_probs / batch_probs.sum(dim=-1, keepdim=True)  # Normalize
            batch_probs = torch.clamp(batch_probs, 0, 1) # Clamp values to be between 0 and 1
            batch_probs = batch_probs / batch_probs.sum(dim=-1, keepdim=True) # Renormalize
            batch_true_probs = batch_probs[brange, true_ids[1:]]
            batch_entropy = torch.distributions.Categorical(probs=batch_probs).entropy()
            batch_embeddings = states[0]

            top_guesses.append(batch_top_guesses.numpy(force=True))
            ranks.append(batch_ranks.numpy(force=True))
            true_probs.append(batch_true_probs.numpy(force=True))
            entropies.append(batch_entropy.numpy(force=True))
            embeddings.append(batch_embeddings.numpy(force=True))

            if batch.size(0) == 1:
                continue
            logits = logits[1:]
            states = states[1:]
            batch = batch[1:]

        true_ids = batch[:, -1]
        brange = list(range(len(true_ids)))
        logits_order = logits[:, -2, :].argsort(descending=True)  # batch x vocab_size
        batch_top_guesses = logits_order[:, 0]
        batch_ranks = torch.eq(logits_order, true_ids.reshape(-1,1).to(device)).nonzero()[:, 1]
        batch_probs = torch.softmax(logits[:, -2, :], dim=-1)
        batch_probs = batch_probs / batch_probs.sum(dim=-1, keepdim=True)  # Normalize
        batch_probs = torch.clamp(batch_probs, 0, 1) # Clamp values to be between 0 and 1
        batch_probs = batch_probs / batch_probs.sum(dim=-1, keepdim=True) # Renormalize
        batch_true_probs = batch_probs[brange, true_ids]
        batch_entropy = torch.distributions.Categorical(probs=batch_probs).entropy()
        batch_embeddings = states[:, -1, :]

        top_guesses.append(batch_top_guesses.numpy(force=True))
        ranks.append(batch_ranks.numpy(force=True))
        true_probs.append(batch_true_probs.numpy(force=True))
        entropies.append(batch_entropy.numpy(force=True))
        embeddings.append(batch_embeddings.numpy(force=True))

  0%|          | 0/1588 [00:00<?, ?it/s]

In [ ]:
df = story_df.copy()
df.loc[1:, 'rank'] = np.concatenate(ranks)
df.loc[1:, 'true_prob'] = np.concatenate(true_probs)
df.loc[1:, 'top_pred'] = np.concatenate(top_guesses)
df.loc[0, 'top_pred'] = autoToken.bos_token_id
df.loc[1:, 'entropy'] = np.concatenate(entropies)
df['embedding'] = [e for e in np.vstack(embeddings)]

non_finite_mask = ~np.isfinite(df['top_pred'])
if non_finite_mask.any():
    print("Warning: Non-finite values found in 'top_pred'. Replacing with unknown token ID.")
    df.loc[non_finite_mask, 'top_pred'] = autoToken.unk_token_id  # Or any other suitable placeholder

df = df.astype({'w_id': 'int32', 'onset': 'float32',
                                      'offset': 'float32', 'token_id': 'int32',
                                      'rank': 'float32', 'true_prob': 'float32',
                                      'top_pred': 'int32', 'entropy': 'float32'}, copy=False)

df['top_pred'] = df.top_pred.apply(autoToken.convert_ids_to_tokens)

print(f"Top-1 accuracy: {(df['rank'] == 0).mean():.3f}")
print(f"Top-10 accuracy: {(df['rank'] < 10).mean():.3f}")

with open('/content/drive/MyDrive/SharedDataonLLM/Results/JHU_MAMBA370_B4_L10.pkl', 'wb') as f:
    pickle.dump(df, f)

Top-1 accuracy: 0.321
Top-10 accuracy: 0.632


In [ ]:
all_stories_gpt_pkl = '/content/drive/MyDrive/SharedDataonLLM/Results/JHU_MAMBA370_B4_L10.pkl'
if exists(all_stories_gpt_pkl):
    with open(all_stories_gpt_pkl, 'rb') as f:
        df = pickle.load(f)

In [ ]:
df2 = df.copy()
df2['diff'] = df2['onset'].diff()

threshold = -100.0
drops = df2[df2['diff'] < threshold]

In [ ]:
x = 0.0
for i in range(df2.shape[0]):
    if df2.at[i, 'diff'] < threshold:
        print(df2.at[i-1, 'onset'])
        print(df2.at[i, 'onset'], '\n')
        x = df2.at[i-1, 'onset']
    df2.at[i, 'onset'] += x

799.81
0.85 

1624.26
8.71 

2032.65
15.19 



In [ ]:
threshold = -100.0
drops = df2[df2['diff'] < threshold]

In [ ]:
df2['onset'] = df2['onset'].fillna(0)
dataframe = df2.copy()
features = 768
duration= 2590
TR=1.5

stim_TRs = np.ceil(duration/TR)
dataframe['TR'] = dataframe.onset.divide(TR).apply(np.floor).apply(int)
words_per_TR      = dataframe.groupby('TR')['word'].apply(list)
embeddings_per_TR = dataframe.groupby('TR')['embedding'].mean()


In [ ]:
words_per_TR

TR
0       [so, i, black, mic, you, know, i, in, and, mau...
1                                        [was, a, junior]
2                                           [in, college]
3                               [when, i, got, my, first]
4                                                [paying]
                              ...                        
1716                                            [just, a]
1717                                       [fraction, of]
1719                                                [her]
1720                               [toughness, toughness]
1721        [of, her, highbridge, highbridge, her, bronx]
Name: word, Length: 1583, dtype: object

In [ ]:
embeddings_per_TR

TR
0       [-0.13932846, 0.09147937, 0.10752694, -0.52368...
1       [-0.51283044, -0.37843838, -0.34866524, 0.1263...
2       [-0.40417707, 0.62267584, -1.1393734, 0.449207...
3       [-0.9944379, 0.15088801, -0.3413588, 0.1330548...
4       [-0.9836641, -1.3662885, -1.6735636, 0.9259765...
                              ...                        
1716    [-0.11926057, 0.3404693, 0.32224745, 0.1809928...
1717    [-0.3777894, -0.12511477, 0.088538565, -0.9113...
1719    [0.47741723, -0.36633506, 0.22698373, 0.170825...
1720    [0.26047987, -0.4577526, 0.5809442, -1.003243,...
1721    [0.2631292, -0.18017948, -0.262976, -0.5015881...
Name: embedding, Length: 1583, dtype: object

black story TRs = 0 - 533 ~ 534 TRs
forgot story TRs = 533 - 1082 ~ 550 TRs

534 + 550 = 1084 ( 0 - 1083 )

piemanpni story TRs = 1088 - 1355 ~ 268 TRs

1084 + 268 = 1352

bronx story TRs = 1365 - 1721 ~ 357 TRs
1352 + 357 = 1709


In [ ]:
dataframe.loc[1655:, 'TR'] += 1

In [ ]:
dataframe.loc[dataframe['story'] == 'black'].reset_index(drop=True)

,w_id,onset,offset,story,word,lemma,PoS_tag_full,token,token_id,rank,true_prob,top_pred,entropy,embedding,diff,TR
0,0,0.240000,0.630000,black,so,so,RB,so,601,NaN,NaN,<|endoftext|>,NaN,"[-0.048636332, -0.22333843, -0.22603035, 0.181...",NaN,0
1,1,0.680000,1.260000,black,i,I,PRP,i,74,32.0,0.003880,",",7.155437,"[0.3199688, -1.1084895, 0.17651823, 0.28267837...",0.440000,0
2,2,1.960000,2.300000,black,was,be,VBD,was,4238,2535.0,0.000031,",",6.667751,"[-0.43351167, 0.08410552, 0.057881575, 0.64090...",1.280000,1
3,3,2.300000,2.450000,black,a,a,DT,a,66,0.0,0.052798,a,6.901180,"[-0.16968352, -0.14271556, -0.06553942, -0.069...",0.340000,1
4,4,2.460000,3.140000,black,junior,junior,NN,jun,30986,12474.0,0.000003,.,5.931690,"[0.19387455, -0.4008525, -0.55816936, 0.098503...",0.160000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1826,1539,790.780029,791.010010,black,what,what,WP,what,5371,2.0,0.031939,this,6.162780,"[-0.33248287, -0.22995368, -0.13772754, 0.2375...",0.210022,528
1827,1540,791.010010,791.130005,black,i,I,PRP,i,74,0.0,0.845012,i,1.309622,"[-0.03616941, -0.08930032, 0.31699958, 0.18955...",0.229980,528
1828,1541,791.130005,791.469971,black,do,do,VB,do,3088,20.0,0.003736,'m,2.428656,"[0.17253925, -0.23621865, 0.030515801, -0.0352...",0.119995,528
1829,1542,792.349976,792.609985,black,thank,thank,NN,thank,47033,237.0,0.000093,i,2.615423,"[0.030501839, 0.11543633, 0.122613996, -0.1586...",1.219971,529


In [ ]:
dataframe.loc[dataframe['story'] == 'forgot'].reset_index(drop=True)

,w_id,onset,offset,story,word,lemma,PoS_tag_full,token,token_id,rank,true_prob,top_pred,entropy,embedding,diff,TR
0,1544,800.659973,0.980000,forgot,this,this,DT,this,2520,38.0,2.435488e-03,for,4.073023,"[-0.24860479, 0.012697011, -0.42084473, -0.214...",-798.960022,534
1,1545,800.789978,1.130000,forgot,is,be,VBZ,is,261,0.0,5.123286e-01,is,3.255076,"[-0.1509966, 0.123737544, 0.014310166, -0.3026...",0.130000,534
2,1546,800.940002,1.200000,forgot,a,a,DT,a,66,2.0,4.068821e-02,my,4.914600,"[-0.12924963, -0.1322417, -0.04133374, -0.3898...",0.150000,534
3,1547,801.010010,1.770000,forgot,story,story,NN,story,19192,520.0,2.478286e-04,job,6.728918,"[0.0375643, 0.04678405, 0.04873924, -0.1449242...",0.070000,535
4,1548,802.020020,2.380000,forgot,that,that,IN,that,3529,0.0,1.153944e-01,that,4.337630,"[-0.12876517, -0.18471737, -0.06970827, 0.2981...",1.010000,535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2571,3674,1618.090088,818.710022,forgot,c,c,NNS,c,68,10125.0,2.188930e-07,out,3.041172,"[0.26458895, 0.2767844, -0.016998734, 0.197662...",1.550049,1079
2572,3675,1619.619995,820.159973,forgot,d,d,NN,d,69,6144.0,6.713222e-06,and,6.942523,"[-0.15522471, 0.08226794, -0.06228528, 0.52149...",1.529968,1080
2573,3676,1621.349976,821.880005,forgot,e,e,NN,e,70,49905.0,1.709112e-09,cd,6.634484,"[-0.18003741, 0.31186178, -0.08267631, -0.2071...",1.729980,1081
2574,3677,1622.780029,823.429993,forgot,f,f,NN,f,71,2021.0,2.724089e-05,fe,6.169029,"[0.51009566, -0.38877687, 0.03360529, 0.282264...",1.429993,1082


In [ ]:
dataframe.loc[dataframe['story'] == 'piemanpni'].reset_index(drop=True)

,w_id,onset,offset,story,word,lemma,PoS_tag_full,token,token_id,rank,true_prob,top_pred,entropy,embedding,diff,TR
0,3679,1632.969971,8.840000,piemanpni,i,I,PRP,i,74,3856.0,0.000003,hi,3.825776,"[0.03996113, 0.3120494, 0.015296619, 0.0373290...",-815.739990,1089
1,3680,1633.250000,9.070000,piemanpni,began,begin,VBD,b,67,11252.0,0.000001,'m,5.236617,"[-0.01710417, 0.07382757, -0.18746616, 0.47919...",0.280000,1089
2,3680,1633.250000,9.070000,piemanpni,began,begin,VBD,egan,30558,1008.0,0.000048,ce,5.253960,"[-0.027335115, -0.570715, -0.0055573657, -0.27...",0.000000,1089
3,3681,1633.329956,9.240000,piemanpni,my,my,PRP$,my,2577,1.0,0.033289,the,6.685097,"[-0.14337721, 0.016274696, -0.11871821, -0.315...",0.080000,1089
4,3682,1633.890015,10.080000,piemanpni,illustrious,illustrious,JJ,ill,408,338.0,0.000325,pr,5.594775,"[0.32825384, 0.046033535, -0.17015441, 0.24915...",0.560000,1090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1240,4666,2031.670044,407.609985,piemanpni,tell,tell,NN,tell,38878,53.0,0.001625,i,3.739511,"[-0.42663386, 0.4074266, 0.12722655, -0.019893...",0.209991,1355
1241,4667,2031.869995,407.769989,piemanpni,me,me,PRP,me,1405,0.0,0.651448,me,2.436560,"[-0.33553562, -0.2745284, 0.06562178, 0.068364...",0.199982,1355
1242,4668,2032.030029,408.000000,piemanpni,all,all,DT,all,455,17.0,0.005571,i,3.879859,"[0.4244389, -0.22334592, -0.0974894, 0.0207444...",0.160004,1355
1243,4669,2032.260010,408.380005,piemanpni,about,about,IN,about,10383,0.0,0.598688,about,2.349410,"[0.19158328, -0.041072845, -0.22298512, -0.194...",0.230011,1355


In [ ]:
dataframe.loc[dataframe['story'] == 'bronx'].reset_index(drop=True)

,w_id,onset,offset,story,word,lemma,PoS_tag_full,token,token_id,rank,true_prob,top_pred,entropy,embedding,diff,TR
0,4671,2047.839966,15.410000,bronx,when,when,WRB,when,9453,29.0,0.004145,and,4.274702,"[-0.09395555, -0.37093642, 0.13364589, 0.18132...",-393.200012,1366
1,4672,2048.060059,15.460000,bronx,i,I,PRP,i,74,0.0,0.497526,i,2.523431,"[-0.06930293, -0.03798423, -0.087007895, 0.139...",0.220000,1366
2,4673,2048.110107,15.660000,bronx,get,get,VB,get,788,37.0,0.003993,went,4.967430,"[-0.59886825, -0.30815792, -0.77506155, -0.207...",0.050000,1366
3,4674,2048.310059,15.760000,bronx,the,the,DT,the,783,5.0,0.039402,there,4.915266,"[-0.43661034, -0.20021273, 0.20637175, -0.0231...",0.200000,1366
4,4675,2048.479980,16.230000,bronx,baton,baton,NN,b,67,2.0,0.020869,be,3.851089,"[-0.030416273, -0.031305663, 0.10452001, -0.03...",0.170000,1366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716,6047,2582.500000,549.859985,bronx,highbridge,NaN,NaN,high,8656,227.0,0.000534,heart,5.389746,"[0.006252462, -0.74319416, -0.16983244, -0.153...",0.219971,1722
1717,6047,2582.500000,549.859985,bronx,highbridge,NaN,NaN,bridge,8298,991.0,0.000019,school,3.554084,"[-0.060384072, 0.19625369, 0.06176495, 0.16984...",0.000000,1722
1718,6048,2582.969971,550.340027,bronx,her,her,PRP$,her,379,2.0,0.043420,and,6.124412,"[-0.24204727, 0.0038755052, -0.0066065863, -0....",0.470032,1722
1719,6049,2582.989990,550.460022,bronx,bronx,Bronx,NNP,bron,40742,423.0,0.000395,heart,6.467436,"[-0.0022244789, -0.12141615, -0.1560114, -0.44...",0.020020,1722


In [ ]:
words_TRs = []
embeddings_TRs = []
stim_TRs = 1721.0

In [ ]:
for tr in np.arange(0, 1084): # 0 - 1084 -> black,forgot >>> 1089 - 1357 -> piemanpni >>> 1366 - 1723 > bronx
    if tr in words_per_TR:
        words_TRs.append(words_per_TR[tr])

        if embeddings_per_TR[tr] is not np.nan:
            embeddings_TRs.append(embeddings_per_TR[tr])
        else:
            embeddings_TRs.append(np.zeros(features))
    else:
        words_TRs.append([])
        embeddings_TRs.append(np.zeros(features))

In [ ]:
for tr in np.arange(1089, 1357): # 0 - 1084 -> black,forgot >>> 1089 - 1357 -> piemanpni >>> 1366 - 1723 > bronx
    if tr in words_per_TR:
        words_TRs.append(words_per_TR[tr])

        if embeddings_per_TR[tr] is not np.nan:
            embeddings_TRs.append(embeddings_per_TR[tr])
        else:
            embeddings_TRs.append(np.zeros(features))
    else:
        words_TRs.append([])
        embeddings_TRs.append(np.zeros(features))

In [ ]:
for tr in np.arange(1366, 1723): # 0 - 1084 -> black,forgot >>> 1089 - 1357 -> piemanpni >>> 1366 - 1723 > bronx
    if tr in words_per_TR:
        words_TRs.append(words_per_TR[tr])

        if embeddings_per_TR[tr] is not np.nan:
            embeddings_TRs.append(embeddings_per_TR[tr])
        else:
            embeddings_TRs.append(np.zeros(features))
    else:
        words_TRs.append([])
        embeddings_TRs.append(np.zeros(features))

In [ ]:
len(embeddings_TRs)

1709

In [ ]:
features = 768
embeddings_TRs_fixed = [np.resize(arr, (features,)) if arr.size > 0 else np.zeros(features) for arr in embeddings_TRs]

embeddings = np.vstack(embeddings_TRs_fixed)
embeddings.shape

(1709, 768)

In [ ]:
X = embeddings
print('X shape: ', embeddings.shape, '\n')
print(X)

X shape:  (1709, 768) 

[[-0.14467816 -0.12470948 -0.05944806 ...  0.14627381 -0.13105941
   0.02467177]
 [-0.08713441 -0.0521846  -0.18191737 ... -0.04276637 -0.29049921
   0.25884104]
 [ 0.09484208  0.06343979 -0.06657428 ... -0.04447627 -0.05743765
  -0.0864915 ]
 ...
 [ 0.10242995 -0.10545719 -0.15944304 ...  0.13329494  0.15545209
   0.00731575]
 [-0.10741489 -0.11638769 -0.01647711 ...  0.22891343 -0.13177045
  -0.00869773]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


In [ ]:
y = parcel_signals
print('y shape: ', y.shape, '\n')
print(y)

y shape:  (1709, 189) 

[[-0.37411946 -0.92605384  0.87349386 ... -0.62784231 -0.42665249
  -0.46386083]
 [ 0.11061479 -1.34038806  1.28639    ... -0.52568405 -0.82295551
  -1.06895736]
 [ 2.05736018  0.77267336  2.25276258 ... -0.27717398 -0.98053248
  -0.99467722]
 ...
 [-0.78809245 -1.44335828 -0.61327838 ...  0.51954664 -0.15943666
  -0.76459508]
 [ 1.48297244  0.44735744  0.60868805 ...  0.85524355  0.85035418
   0.43574067]
 [ 1.90801474  1.52727157  2.12196952 ...  1.57311847  1.88082379
   1.66044385]]


In [ ]:
external = KFold(n_splits=2)
internal = KFold(n_splits=5)


stdScaler = StandardScaler(with_mean=True, with_std=True)

delayer = Delayer(delays=[2,3,4,5])

alphas = np.logspace(1,10,10)
Ridge = KernelRidgeCV(alphas=alphas, cv=internal)

pipeline = make_pipeline(stdScaler, delayer, Ridge)

In [ ]:
y_pred = []

for train, test in external.split(y):
    pipeline.fit(X[train], y[train])
    predicted = pipeline.predict(X[test])
    y_pred.append(predicted)

y_pred_V = np.vstack(y_pred)

In [ ]:
y_pred_V.shape

(1709, 189)

In [ ]:
all_story_r_score = correlation_score(y, y_pred_V)
print("Mean r score", all_story_r_score.mean())
print("Max r score", all_story_r_score.max())
print("Min r score", all_story_r_score.min())

Mean r score 0.014811020481725402
Max r score 0.15882875150441708
Min r score -0.05796064378276013


In [ ]:
all_story_r_score = correlation_score(y, y_pred_V)
print("Mean r score", all_story_r_score.mean())
print("Max r score", all_story_r_score.max())
print("Min r score", all_story_r_score.min())

Mean r score 0.015434918493756696
Max r score 0.16410784083867366
Min r score -0.04674265192533912


In [ ]:
all_story_r_score = correlation_score(y, y_pred_V)
print("Mean r score", all_story_r_score.mean())
print("Max r score", all_story_r_score.max())
print("Min r score", all_story_r_score.min())

Mean r score 0.019389316781762825
Max r score 0.18840199917792028
Min r score -0.03611359231580105


In [ ]:
all_story_r_score = correlation_score(y, y_pred_V)
print("Mean r score", all_story_r_score.mean())
print("Max r score", all_story_r_score.max())
print("Min r score", all_story_r_score.min())

Mean r score 0.02264861920348397
Max r score 0.18669490875315795
Min r score -0.033634958646951735


In [ ]:
all_story_r_score = correlation_score(y, y_pred_V)
print("Mean r score", all_story_r_score.mean())
print("Max r score", all_story_r_score.max())
print("Min r score", all_story_r_score.min())

In [ ]:
all_story_r_score = correlation_score(y, y_pred_V)
print("Mean r score", all_story_r_score.mean())
print("Max r score", all_story_r_score.max())
print("Min r score", all_story_r_score.min())

Mean r score 0.015036224865406372
Max r score 0.15230405838391767
Min r score -0.07726050467562469


In [ ]:
img = mask.inverse_transform(all_story_r_score)
img.to_filename('/content/drive/MyDrive/SharedDataonLLM/Results/sub-127_AS_JHU_MAMBA370_B4_L10.nii.gz')